### LOAD LIBRARIES

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras.layers import Dense,Dropout
from keras.models import Model 
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
import numpy as np
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, AveragePooling2D, Flatten,Conv2D,MaxPooling2D
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import Callback,ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

### PROCESS IMAGES

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=True, 
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=True)
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

# GIVE THE PATH OF THE TRAIN AND TEST IMAGES 
train_generator = datagen.flow_from_directory('.../train/images/', target_size=(128,128), class_mode='categorical', batch_size=128)

# val_generator = datagen.flow_from_directory('D:/output/val/', target_size=(128,128), class_mode='categorical', batch_size=128)
# test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)
test_generator = datagen.flow_from_directory('.../output/test/', target_size=(128,128), class_mode='categorical', batch_size=128)


### CNN

In [ ]:

#Define a Convolutional Neural Network Model

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (128,128,3)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(19, activation='softmax'))

learning_rate = 0.001

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(learning_rate),
              metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

model.summary()
history= model.fit(train_generator,epochs=25)


In [ ]:
### SAVE THE RESULTS IN A CSV FILE

In [ ]:
prediction=model.predict(test_generator, steps=len(test_generator))
prediction.shape
prediction_index= np.argmax(prediction, axis=1)
prediction_index.shape
labels=train_generator.class_indices
print(type(labels), len(labels))
labels = dict((value,key) for key,value in labels.items())
# labels
predicted_class = [labels[k] for k in prediction_index]
len(predicted_class)
# predicted_class

filenames_ = test_generator.filenames
filenames_

filenames=[]

for e in filenames_:
    e = e[7:]
    filenames.append(e)
#     print(filenames)

results = pd.DataFrame({"file_name":filenames})
results.head()

results = pd.DataFrame({"file_name":filenames, "category":predicted_class})
results.head()

results.to_csv('model_cnn.csv', index=False)
# model.save('cnn.keras')

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.xlabel('No. of epochs')
plt.ylabel('accuracies')
plt.legend(['Train','Validation'])
plt.show()

plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.xlabel('No. of epochs')
plt.ylabel('Losses')
plt.legend(['Train','Validation'])
plt.show()

### TUNED CNN (BASE MODEL PERFORM BETTER)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (128,128,3)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(19, activation='softmax'))

learning_rate = 0.001

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(learning_rate),
              metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

model.summary()
history= model.fit(train_generator,epochs=30)
prediction=model.predict(test_generator, steps=len(test_generator))
prediction.shape
prediction_index= np.argmax(prediction, axis=1)
prediction_index.shape
labels=train_generator.class_indices
print(type(labels), len(labels))
labels = dict((value,key) for key,value in labels.items())
# labels
predicted_class = [labels[k] for k in prediction_index]
len(predicted_class)
# predicted_class

filenames_ = test_generator.filenames
filenames_

filenames=[]

for e in filenames_:
    e = e[7:]
    filenames.append(e)
#     print(filenames)

results = pd.DataFrame({"file_name":filenames})
results.head()

results = pd.DataFrame({"file_name":filenames, "category":predicted_class})
results.head()

results.to_csv('cnn_tuned.csv', index=False)